In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from pytorch_lightning import LightningModule

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from model import SimpleGaussianDiffusionModel

# Пример простой модели с добавлением гауссовского шума и constant scheduler
# Допустим, у нас есть простой автоэнкодер или просто сетка из линейных слоёв,
# на вход которой подаются данные, к которым добавляется шум.
# На выходе модель должна приблизиться к исходным данным.

class SimpleGaussianDiffusionModel(LightningModule):
    def __init__(self, input_dim=100, hidden_dim=64, lr=1e-4, weight_decay=1e-5, noise_std=0.1):
        super().__init__()
        self.save_hyperparameters()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim, input_dim),
        )
        self.noise_std = noise_std
        self.lr = lr
        self.weight_decay = weight_decay

    def forward(self, x):
        # Проход через простую сеть
        h = self.encoder(x)
        return self.decoder(h)

    def training_step(self, batch, batch_idx):
        x, _ = batch
        # Добавляем гауссовский шум к входным данным
        noise = torch.randn_like(x) * self.noise_std
        noisy_x = x + noise
        pred = self(noisy_x)
        loss = ((pred - x) ** 2).mean()
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # Constant scheduler: без изменения learning rate
        optimizer = optim.AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)
        return optimizer


In [4]:



# Предположим, что data/weights.npy и data/good_paths.npy имеют одинаковую длину
# и что данные представлены в формате [кол-во_примеров, размер_входа].
# Например, если вход - это вектор длины input_dim, то веса и пути должны быть [N, input_dim].
# Если это матрицы [N, M, M], то вы можете их расплющить до [N, M*M] или модифицировать модель.

class GraphDataset(Dataset):
    def __init__(self, weights_path, good_paths_path):
        self.weights = np.load(weights_path)        # shape: [1000, 30, 30]
        self.good_paths = np.load(good_paths_path)  # shape: [1000, 30]
        
        self.weights = torch.from_numpy(self.weights).float()
        self.good_paths = torch.from_numpy(self.good_paths).float()

    def __len__(self):
        return len(self.weights)

    def __getitem__(self, idx):
        return self.weights[idx], self.good_paths[idx]

In [5]:
# Загрузка данных
train_dataset = GraphDataset( "data/weights.npy",  "data/good_paths.npy")
# Создание DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
# Инициализация модели
# Предположим, что размер входа соответствует форме данных.
# Если weights.npy имеет форму [N, D], то input_dim = D
input_dim = train_dataset.weights.shape[1]
model = SimpleGaussianDiffusionModel(input_dim=input_dim, hidden_dim=128, lr=1e-4, weight_decay=1e-5, noise_std=0.1)
# Перепишем training_step модели так, чтобы использовать good_paths как цель:
# Для этого переопределим метод в наследнике:
class ModelWithTargets(SimpleGaussianDiffusionModel):
    def training_step(self, batch, batch_idx):
        weights, good_paths = batch
        noise = torch.randn_like(weights) * self.noise_std
        noisy_input = weights + noise
        pred = self(noisy_input)
        loss = ((pred - good_paths) ** 2).mean()
        self.log("train_loss", loss, prog_bar=True)
        return loss
model = ModelWithTargets(input_dim=input_dim, hidden_dim=128, lr=1e-4, weight_decay=1e-5, noise_std=0.1)
# Настройка тренера
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
)
# Обучение
trainer.fit(model, train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\revya\.conda\envs\difusco\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 4.0 K  | train
1 | decoder | Sequential | 3.9 K  | train
-----------------------------------------------
7.8 K     Trainable params
0         Non-trainable params
7.8 K     Total params
0

Epoch 0:   0%|          | 0/16 [00:00<?, ?it/s] 

RuntimeError: DataLoader worker (pid(s) 19240, 21828, 23592, 1056) exited unexpectedly